# Deploy RigoChat-7b Model Package from AWS Marketplace 


**RigoChat-7b** is the second version of RigoChat, a family of Large Language Models (LLMs) designed to solve typical NLP tasks with Spanish instructions such as: Tool Use, Summarization, Math, Code, Abstractive-QA, etc. This model has no specific use case and can be applied to a wide range of tasks. Indeed, it offers an improvement for generalist tasks in Spanish, particularly in RAG (Retriever Augmented Generation) systems with Spanish databases, as its training focused on resolving questions about contexts to prevent hallucinations and ensure safe responses. 


This sample notebook shows you how to deploy [RigoChat-7b]() using Amazon SageMaker.

## Pre-requisites:
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**
1. To deploy this ML model successfully, ensure that:
    1. Either your IAM role has these three permissions and you have authority to make AWS Marketplace subscriptions in the AWS account used: 
        1. **aws-marketplace:ViewSubscriptions**
        1. **aws-marketplace:Unsubscribe**
        1. **aws-marketplace:Subscribe**  
    2. or your AWS account has a subscription to [RigoChat-7b](). If so, skip step: [Subscribe to the model package](#1.-Subscribe-to-the-model-package)

## Contents:
1. [Subscribe to the model package](#1.-Subscribe-to-the-model-package)
2. [Create an endpoint and perform real-time inference](#2.-Create-an-endpoint-and-perform-real-time-inference)
   1. [Create an endpoint](#A.-Create-an-endpoint)
   2. [Create input payload](#B.-Create-input-payload)
   3. [Perform real-time inference](#C.-Perform-real-time-inference)
   4. [Visualize output](#D.-Visualize-output)
   5. [Delete the endpoint](#E.-Delete-the-endpoint)
3. [Perform batch inference](#3.-Perform-batch-inference) 
4. [Clean-up](#4.-Clean-up)
    1. [Delete the model](#A.-Delete-the-model)
    2. [Unsubscribe to the listing (optional)](#B.-Unsubscribe-to-the-listing-(optional))
    

## Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

## 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page [RigoChat-7b]().
2. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
3. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
4. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

In [ ]:
model_package_arn = "arn:aws:sagemaker:eu-west-1:<AWS-ACCOUNT>:model-package/RigoChat-7b"

In [ ]:
import json

import boto3
import sagemaker as sage
from sagemaker import ModelPackage, get_execution_role

from src.utils import TextGenerationPredictor

In [ ]:
role = get_execution_role()

sagemaker_session = sage.Session()

bucket = sagemaker_session.default_bucket()
runtime = boto3.client("runtime.sagemaker")
bucket

## 2. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

In [ ]:
model_name = "rigochat-v2"

content_type = "application/json"

real_time_inference_instance_type = "ml.g5.2xlarge"

batch_transform_inference_instance_type = "ml.g5.4xlarge"

### A. Create an endpoint

In [ ]:
# create a deployable model from the model package.
model = ModelPackage(
    role=role, model_package_arn=model_package_arn, sagemaker_session=sagemaker_session
)

# Deploy the model
model.deploy(1, real_time_inference_instance_type, endpoint_name=model_name)

Once endpoint has been created, you would be able to perform real-time inference.

### B. Create input payload

In [ ]:
with open("data/input/text_generation_messages_input.json") as fp:
    payload = json.load(fp)
payload

### C. Perform real-time inference
Use the `TextGenerationPredictor` class from provided `utils.py` to easily handle responses.

Base `sagemaker.huggingface.model.HuggingFacePredictor` is also a viable alternative for non-streaming predictions

In [ ]:
predictor = TextGenerationPredictor(endpoint_name=model_name, sagemaker_session=sagemaker_session)
response = predictor.predict(payload)

### D. Visualize output

In [ ]:
choices = response["choices"]
print(choices)
print(choices[0]["message"]["content"])

### D.2 Stream real-time inference output

The method `TextGenerationPredictor.stream` returns a response stream iterator which allows to process and visualize model output in real time.

See [here](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_runtime_InvokeEndpointWithResponseStream.html) and [here](https://docs.aws.amazon.com/sagemaker/latest/dg/realtime-endpoints-test-endpoints.html#test-invoke-endpoint-with-response-stream) for handling streaming with standard predictor classes.

In [ ]:
predictor = TextGenerationPredictor(endpoint_name=model_name, sagemaker_session=sagemaker_session)
response_stream = predictor.stream(payload)

for chunk in response_stream:
  content = chunk["choices"][0]["delta"].get("content", "")
  print(content, end='')

### E. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [ ]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

## 3. Perform batch inference

In this section, you will perform batch inference using multiple input payloads together. If you are not familiar with batch transform, and want to learn more, see these links:
1. [How it works](https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-batch-transform.html)
2. [How to run a batch transform job](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-batch.html)

In [ ]:
transform_input_data = "data/input/text_generation_messages_input.json"

with open(transform_input_data) as fp:
    payload = json.load(fp)
payload

In [ ]:
# upload the batch-transform job input files to S3
transform_input = sagemaker_session.upload_data(transform_input_data, key_prefix=model_name)
print("Transform input uploaded to " + transform_input)

In [ ]:
# Run the batch-transform job
transformer = model.transformer(1, batch_transform_inference_instance_type)
transformer.transform(transform_input, content_type=content_type)
transformer.wait()

In [ ]:
# output is available on following path
transformer.output_path

In [ ]:
!aws s3 cp $transformer.output_path . --recursive

output_path = f"{transform_input_data.rsplit('/')[-1]}.out"
with open(output_path, "r") as fp:
    response = json.load(fp)

In [ ]:
choices = response["choices"]
print(choices)
print(choices[0]["message"]["content"])

## 4. Clean-up

### A. Delete the model

In [ ]:
model.delete_model()

### B. Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

